In [ ]:
from collections import namedtuple, defaultdict, Counter
from glob import glob
from itertools import groupby
from operator import itemgetter
from os import remove
from pathlib import Path
import pickle

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel as LDA
from gensim.models import AuthorTopicModel as ATM

from matplotlib import pyplot as plt

import numpy as np

from pandas import DataFrame, read_csv, concat

import pyLDAvis as ldavis
import pyLDAvis.gensim

import seaborn as sns

from sklearn.model_selection import train_test_split

import scipy
from scipy.spatial.distance import cdist
from scipy.sparse import coo_matrix as sparse_matrix

ldavis.enable_notebook()
%matplotlib notebook
#%precision 4


Candidate = namedtuple('Candidate', ['iterations', 'num_topics'])



def get_i_t(filename):
    _, content, document_type = filename.split('-')
    i, t, _ = content.split('_')
    return int(i[1:]), int(t[1:]), document_type.split('.')[0]

get_texts = lambda df: df[target].str.split()
tobows = lambda df, d: concat([df['Anomaly_ID'], get_texts(df).apply(d.doc2bow)], axis=1)



isr2 = 2.0 ** -.5

def hellinger(x, y):
    return isr2 * np.sqrt(((np.sqrt(x) - np.sqrt(y)) ** 2).sum())



report_types = 'ISA', 'PFR', 'DPFR'

TEST_SIZE = 0.3
min_occurances = 2
target = 'GLOMUNSTEM'

%matplotlib notebook
%precision 4

BASEDIR = Path('../data')
OUT = Path('../output/atm/')



In [ ]:
with open(BASEDIR / 'processed_authors.csv') as fd:
    af = read_csv(fd)
af.shape
af.head()

In [ ]:
authors_by_type = {
    t: af[af.ReportType == t] 
    for t in af.ReportType.unique()
}

In [ ]:
documents_by_type = {
    t: read_csv(OUT / f'../norm_{t}.csv').dropna()
    for t in af.ReportType.unique()
}

In [ ]:
def author_document_downselect(documents, authors, ANOMALY_LABEL='Anomaly_ID'):
    documents[ANOMALY_LABEL] = \
      'A' + documents[ANOMALY_LABEL].apply(str)

    idx = documents[ANOMALY_LABEL].isin(
        authors[ANOMALY_LABEL].unique()
    )

    documents = documents[idx]
    
    idx = authors[ANOMALY_LABEL].isin(
        documents[ANOMALY_LABEL].unique()
    )
    authors = authors[idx]
    return documents, authors


for document_type in documents_by_type:
    documents_by_type[document_type], authors_by_type[document_type] = \
        author_document_downselect(
          documents_by_type[document_type],
          authors_by_type[document_type]
    )


In [ ]:
documents_by_type['ISA'].head()

In [ ]:
def get_train_test_and_vocab(documents, test_size, min_occurances=min_occurances):
    train, test = train_test_split(
        documents, test_size=test_size
    )
    train.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    vocab = Dictionary(
        train[target].str.split()
    )
    vocab.filter_extremes(no_below=min_occurances)

    return train, test, vocab


In [ ]:
model_by_type, train_documents_by_type, test_documents_by_type, dictionary_by_type = \
  dict(), dict(), dict(), dict()

train_author_table_by_type = dict()
test_author_table_by_type = dict()

def attribution_table(documents, relevent_authors):
    store = defaultdict(set)
    for idx, anomaly in documents.iterrows():
        authors_documents = relevent_authors[
            relevent_authors.Anomaly_ID == anomaly.Anomaly_ID
        ]

        for author in authors_documents.Users_ID:
            store[author].add(idx)

    return {k: list(v) for k, v in store.items()}


for document_type in documents_by_type:
    train_documents_by_type[document_type], \
    test_documents_by_type[document_type], \
    dictionary_by_type[document_type] = \
        get_train_test_and_vocab(documents_by_type[document_type], TEST_SIZE)

    train_author_table_by_type[document_type] = attribution_table(
        train_documents_by_type[document_type],
        authors_by_type[document_type]
    )

    test_author_table_by_type[document_type] = attribution_table(
        test_documents_by_type[document_type],
        authors_by_type[document_type]
    )

    print(f'{document_type: <4}')

In [ ]:
election_by_type = {
    'ISA' : Candidate(iterations=351, num_topics=40),
    'PFR' : Candidate(iterations=251, num_topics=25),
    'DPFR': Candidate(iterations=351, num_topics=25),
}

for document_type in election_by_type:
    c = election_by_type[document_type]

    corpus = tobows(
        train_documents_by_type[document_type],
        dictionary_by_type[document_type]
    )[target]

    atm = ATM(
        corpus=list(corpus),
        author2doc=train_author_table_by_type[document_type],
        num_topics=c.num_topics,
        iterations=c.iterations,
    )
    model_by_type[document_type] = atm


In [ ]:
def get_document_topics(model, doc_bow):

    gamma_chunk, sstats = model.inference(
        chunk=[doc_bow], author2doc=dict(), doc2author=dict(), 
        rhot=1.00,
        collect_sstats=True
    )

    return gamma_chunk


def get_model_author_topic_vectors(model):
    author_topic_vectors = np.zeros(
        (model.num_authors, model.num_topics)
    )

    for i, author in enumerate(model.id2author.values()):
        idx, scores = zip(*model.get_author_topics(author))
        author_topic_vectors[i, idx] = scores

    return author_topic_vectors


def get_sorted_authors(model, doc_bow, author_topic_vectors, metric=hellinger):
    doc_vector = get_document_topics(model, doc_bow)

    author_scores = np.argsort(
        cdist(doc_vector, author_topic_vectors, metric=metric)
    )

    contenders = [
        model.id2author[idx]
        for idx in author_scores[0]
    ]

    return contenders


def get_all_ranks_and_counts(model, publication_counts, authors, dictionary, documents):
    author_topic_vectors = get_model_author_topic_vectors(model)

    for idx, row in tobows(documents, dictionary).iterrows():
        Anomaly_ID, doc_bow = row['Anomaly_ID'], row[target]

        contenders = get_sorted_authors(model, doc_bow, author_topic_vectors)
        
        real_authors = [
            a
            for a in authors
            if idx in authors[a]
        ]

        for a in real_authors:
            try:
                rank = contenders.index(a)
            except ValueError as e:
                rank = 500   

            try:
                publications = publication_counts[a]
            except KeyError:
                publications = 0

            yield publications, rank
            break


In [ ]:
# train_author_table_by_type

TT = 'ISA'

publication_counts_by_type = dict()
for t in train_author_table_by_type:
    _authors = train_author_table_by_type[t]
    publication_counts_by_type[t] = {a: len(_authors[a]) for a in _authors}
    del _authors

train = list(get_all_ranks_and_counts(
    model_by_type[TT],
    publication_counts_by_type[TT],
    train_author_table_by_type[TT],
    dictionary_by_type[TT],
    train_documents_by_type[TT]
))

test = list(get_all_ranks_and_counts(
    model_by_type[TT],
    publication_counts_by_type[TT],
    test_author_table_by_type[TT],
    dictionary_by_type[TT],
    test_documents_by_type[TT]
))

In [ ]:
print(train_documents_by_type[TT].shape)
print(train_documents_by_type[TT].iloc[0][target])

from itertools import chain
#set(chain(*train_author_table_by_type['ISA'].values()))
#set(chain(*test_author_table_by_type['ISA'].values()))


In [ ]:
def plot_arc(arc):
    _ = plt.figure()
    # Load the dataset
    data = DataFrame(columns=["publications", "rank"], data=arc)
    _ = sns.jointplot(x=data['publications'], y=data['rank'], 
                      kind='hex', 
                      xlim=(0, 250), ylim=(0, 300), gridsize=30
                     )

_ = plot_arc(train)

_ = plot_arc(test)

In [ ]:
def plot(arc, label, top):
    _ = plt.figure()
    # Load the dataset
    ranks = np.array(arc, dtype=np.float)[:,1]
    #print(ranks)
    
    def get_stats(ranks):
        for cap in range(500):
            # there are no negatives
            idx = ranks < cap
            accuracy = np.sum(idx) / len(idx) * 100

            yield cap, accuracy
    x, y = zip(*list(get_stats(ranks)))
    plt.scatter(x[1:], [k-j for j, k in zip(y, y[1:])], alpha=.1)
    plt.axvline(x=top, color='r')
    plt.title(f'{label} {y[top]:.2f}% at K={top}')
    plt.xlabel('K')
    plt.ylabel('improved recall at K')

    
top = 15
plot(train, f'{TT}-train', top)
plot(test, f'{TT}-test', top)

In [ ]:
top = 2
keys = np.array(test, dtype=np.float)[:,1] <= top
keys.sum()
QUERY_DOCS = test_documents_by_type[TT][keys]
L = QUERY_DOCS.GLOMUNSTEM.apply(lambda x: len(str.split(x)))
L.median(), L.mean(), L.size

In [ ]:
QUERY_DOCS.head()

In [ ]:
from random import sample

def subsample_bow(bow, size, iterations):
    doc = []
    for key, count in bow:
        doc.extend([key for _ in range(count)])
    
    for _ in range(iterations):
        if size >= len(doc):
            continue
        yield list(Counter(sample(doc, size)).items())


def query_strength_investigation(
    model, authors, dictionary, documents,
    iterations, sample_sizes
):
    author_topic_vectors = get_model_author_topic_vectors(model)

    for idx, row in tobows(documents, dictionary).iterrows():
        Anomaly_ID, doc_bow = row['Anomaly_ID'], row[target]

        for size in sample_sizes:
            for new_bow in subsample_bow(doc_bow, size, iterations):

                contenders = get_sorted_authors(
                    model, new_bow, author_topic_vectors
                )

                real_authors = [
                    a
                    for a in authors
                    if idx in authors[a]
                ]

                for a in real_authors:
                    try:
                        rank = contenders.index(a)
                    except ValueError as e:
                        rank = 500

                    yield size, rank


In [ ]:
Q = list(query_strength_investigation(
    model_by_type[TT],
    test_author_table_by_type[TT],
    dictionary_by_type[TT],
    QUERY_DOCS, 100, list(range(4, 100, 5))
))

In [ ]:
x, y = zip(*sorted(Q, key=itemgetter(0)))

_ = plt.figure()
_ = sns.boxplot(
    x=x, y=y, 
    palette="Set3")
plt.xticks(rotation=70)
plt.ylim((0, 25))
plt.axhline(y=5, color='r')
plt.axhline(y=10, color='r')
plt.axhline(y=15, color='r')
plt.title('Good Test Document Subset Investigation')
plt.xlabel('Random subset # of critical Words')
_ = plt.ylabel('New Ranking from document with R <=2')

In [ ]:
#QUERY_DOCS.GLOMUNSTEM.apply(lambda s: len(s.split())).argsort()


In [ ]:
elm = 10
QUERY_DOCS.GLOMUNSTEM.loc[elm]
#QUERY_DOCS.GLOMPRE.loc[elm]

In [ ]:
keys = \
    (np.array(test, dtype=np.float)[:,1] >=10) & \
    (np.array(test, dtype=np.float)[:,1] <= 20)

keys.sum()
MID_QUERY_DOCS = test_documents_by_type[TT][keys]
L = MID_QUERY_DOCS.GLOMUNSTEM.apply(lambda x: len(str.split(x)))
L.median(), L.mean(), L.size

In [ ]:
MID_QUERY_DOCS.head()

In [ ]:
R = list(query_strength_investigation(
    model_by_type[TT],
    test_author_table_by_type[TT],
    dictionary_by_type[TT],
    MID_QUERY_DOCS, 100, list(range(4, 100, 5))
))

In [ ]:
x, y = zip(*sorted(R, key=itemgetter(0)))

_ = plt.figure()
_ = sns.boxplot(
    x=x, y=y,
    palette="Set3")
plt.ylim((0, 80))
plt.xticks(rotation=70)
plt.axhline(y=20, color='r')
#plt.axhline(y=10, color='r')
#plt.axhline(y=15, color='r')
plt.title('Mediocore Test Document Subset Investigation')
plt.xlabel('Random subset # of critical Words')
_ = plt.ylabel('New Ranking from document with (10 <= R <=20)')

In [ ]:
keys = \
    (np.array(test, dtype=np.float)[:,1] > 100) & \
    (np.array(test, dtype=np.float)[:,1] < 500)


BAD_QUERY_DOCS = test_documents_by_type[TT][keys]
display(BAD_QUERY_DOCS.GLOMPRE.iloc[0])
display(BAD_QUERY_DOCS[target].iloc[0])
